# Make Classes for the Microplates in the Strateos catalog



In [1]:
import pandas as pd
from typing import List, Union, Optional, Tuple
import rdflib
from rdflib import Graph, Namespace, Literal, URIRef, BNode
from rdflib.namespace import RDF, RDFS, OWL, XSD
from urllib.parse import quote


In [2]:
# Type declaration
Triple = Tuple[URIRef, URIRef, URIRef]

In [3]:
workbook = pd.ExcelFile('../owl/Strateos containers.xlsx',engine='openpyxl')
strateos_catalog = pd.read_excel(workbook, "Strateos Containers")
strateos_plates = strateos_catalog[strateos_catalog['Well Count'] > 1]
strateos_plates.head()


,Id,Vendor,Catalog Number,Name,Retired At,Well Count,Well Depth (mm),Height (mm),Well Volume (ul),Column Count,Capabilities,Acceptable Lids,Sale Price
0,384-pcr,Eppendorf,951020539,384-Well twin.tec PCR Plate,-,384,9.60,10.6,40.0,24,"bluewash, dispense-destination, echo_dest, env...","ultra-clear, foil",*$9.35*
1,96-10-spot-vplex-m-pro-inflamm1-MSD,Mesoscale,K15048G,96-well 10-spot v-plex mouse pro-inflammatory ...,-,96,11.68,14.35,410.0,12,"bluewash, cover, deseal, dispense-destination,...","low_evaporation, standard, universal, ultra-clear",*$0.00*
4,384-flat-white-white-nbs,Corning,3574,384-Well Low Flange White FlatBottom Polystyre...,-,384,11.43,14.35,80.0,24,"cover, deseal, dispense-destination, echo_dest...","universal, standard, ultra-clear, foil",*$11.80*
5,384-flat-white-white-optiplate,PerkinElmer,6007290,384-Well White Opaque Microplate,-,384,10.45,14.4,105.0,24,"bluewash, cover, dispense-destination, echo_de...","universal, ultra-clear, foil",*$8.28*
6,96-v-kf,Fisher,22-387-030,96-well KingFisher PCR microplate,-,96,12.90,14.5,200.0,12,"cover, dispense-destination, envision, image_p...",standard,*$6.31*


In [4]:
for i, x in strateos_plates.iterrows():
    break
print(x)
print(x.index)
x['Well Depth (mm)']

Id                                                            384-pcr
Vendor                                                      Eppendorf
Catalog Number                                              951020539
Name                                      384-Well twin.tec PCR Plate
Retired At                                                          -
Well Count                                                        384
Well Depth (mm)                                                   9.6
Height (mm)                                                      10.6
Well Volume (ul)                                                 40.0
Column Count                                                       24
Capabilities        bluewash, dispense-destination, echo_dest, env...
Acceptable Lids                                     ultra-clear, foil
Sale Price                                                    *$9.35*
Name: 0, dtype: object
Index(['Id', 'Vendor', 'Catalog Number', 'Name', 'Retired At', 'Wel

9.6

In [5]:
CATALOG_IRI = URIRef('urn:absolute:strateos-containers')
ENTRY_IRI = URIRef('urn:absolute:strateos_containers_catalog')

In [6]:
g = Graph(identifier=CATALOG_IRI)

In [7]:
co_ns = Namespace('https://www.dropbox.com/s/s1e2dzw64m01f9n/container-ontology.ttl#')
strateos_ns = Namespace(CATALOG_IRI + '#')
strateos_entries_ns = Namespace(ENTRY_IRI + '#')
om_ns = Namespace('http://www.ontology-of-units-of-measure.org/resource/om-2/')
g.bind('cont', co_ns)
g.bind('cat', strateos_ns)
g.bind('om', om_ns)
g.bind('rdf', RDF)
g.bind('rdfs', RDFS)
g.bind('owl', OWL)
g.bind('cat_ent', strateos_entries_ns)
g.base = CATALOG_IRI

In [8]:
list(g.namespace_manager.namespaces())

[('xml', rdflib.term.URIRef('http://www.w3.org/XML/1998/namespace')),
 ('rdf', rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#')),
 ('rdfs', rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#')),
 ('xsd', rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#')),
 ('cont',
  rdflib.term.URIRef('https://www.dropbox.com/s/s1e2dzw64m01f9n/container-ontology.ttl#')),
 ('cat', rdflib.term.URIRef('urn:absolute:strateos-containers#')),
 ('om',
  rdflib.term.URIRef('http://www.ontology-of-units-of-measure.org/resource/om-2/')),
 ('owl', rdflib.term.URIRef('http://www.w3.org/2002/07/owl#')),
 ('cat_ent', rdflib.term.URIRef('urn:absolute:strateos_containers_catalog#'))]

In [9]:
help(g.namespace_manager)

Help on NamespaceManager in module rdflib.namespace object:

class NamespaceManager(builtins.object)
 |  NamespaceManager(graph)
 |  
 |  Class for managing prefix => namespace mappings
 |  
 |  Sample usage from FuXi ...
 |  
 |  .. code-block:: python
 |  
 |      ruleStore = N3RuleStore(additionalBuiltins=additionalBuiltins)
 |      nsMgr = NamespaceManager(Graph(ruleStore))
 |      ruleGraph = Graph(ruleStore,namespace_manager=nsMgr)
 |  
 |  
 |  and ...
 |  
 |  .. code-block:: pycon
 |  
 |      >>> import rdflib
 |      >>> from rdflib import Graph
 |      >>> from rdflib.namespace import Namespace, NamespaceManager
 |      >>> exNs = Namespace('http://example.com/')
 |      >>> namespace_manager = NamespaceManager(Graph())
 |      >>> namespace_manager.bind('ex', exNs, override=False)
 |      >>> g = Graph()
 |      >>> g.namespace_manager = namespace_manager
 |      >>> all_ns = [n for n in g.namespace_manager.namespaces()]
 |      >>> assert ('ex', rdflib.term.URIRef('http:/

In [10]:
isinstance(co_ns, str)

True

In [23]:
CATALOG_IRI

rdflib.term.URIRef('urn:absolute:strateos-containers')

In [11]:
g.add((CATALOG_IRI, RDF.type, OWL.ontology))
for x in [co_ns, om_ns]:
    g.add((CATALOG_IRI, OWL.imports, URIRef(x.rstrip("#"))))
g.add((CATALOG_IRI, RDFS.label, Literal("Strateos Catalog Containers", lang='en')))

In [12]:

def add_vendor(vendor_string, graph=g) -> URIRef:
    vendor_iri = co_ns[quote(vendor_string)]
    vendor_name = Literal(vendor_string)    
    graph.add((vendor_iri, RDF.type, co_ns.VendorFirm))
    graph.add((vendor_iri, RDFS.label, vendor_name))
    graph.add((vendor_iri, co_ns.vendorName, vendor_name))
    return vendor_iri

def find_vendor(vendor_string, graph=g) -> Optional[URIRef]:
    vendor_iri = co_ns[quote(vendor_string)]
    if (vendor_iri, RDF.type, co_ns.VendorFirm) in graph:
        return vendor_iri
    return None

In [13]:
def catalog_entry(vendor: str, catalog_number: Union[int, str], graph=g) -> URIRef:
    if isinstance(catalog_number, int):
        catalog_number = str(catalog_number)
    assert isinstance(catalog_number, str)
    vendor_iri = find_vendor(vendor, graph)
    new_iri = strateos_entries_ns[quote(vendor + catalog_number)]
    graph.add((new_iri, RDF.type, co_ns.CatalogEntry))
    graph.add((new_iri, co_ns.hasVendor, vendor_iri))
    graph.add((new_iri, co_ns.catalogNumber, Literal(catalog_number)))
    return new_iri
    

In [14]:
class WellDepth(URIRef):
    depth: float
    unit: URIRef
    count = 0
    def __new__(cls, *args, **kwargs):
        ns = kwargs['ns']
        URI = ns[f"wellDepth{cls.count}"]
        cls.count += 1
        return super().__new__(cls, URI)
    
    def __init__(self, depth: float, unit=om_ns.millimetre, ns=strateos_ns):
        self.depth = depth
        self.unit = unit
        super().__init__()
        
    def triples(self) -> List[Tuple[URIRef, URIRef, URIRef]]:
        return [
            (self, RDF.type, om_ns.Depth),
            (self, om_ns.hasNumericalValue, Literal(self.depth, datatype=XSD.float)),
            (self, om_ns.hasDimension, self.unit),
        ]
        

def make_well_depth(depth: float, unit=om_ns.millimetre, ns=strateos_ns, graph=g) -> URIRef:
    depth_uri = WellDepth(depth, unit=unit, ns=ns)
    for triple in depth_uri.triples():
        graph.add(triple)
    return depth_uri
    
    

In [15]:
class Height(URIRef):
    height: float
    unit: URIRef
    count = 0
    def __new__(cls, *args, **kwargs):
        ns = kwargs['ns']
        URI = ns[f"height{cls.count}"]
        cls.count += 1
        return super().__new__(cls, URI)
    
    def __init__(self, height: float, unit=om_ns.millimetre, ns=strateos_ns):
        self.height = height
        self.unit = unit
        super().__init__()
        
    def triples(self) -> List[Tuple[URIRef, URIRef, URIRef]]:
        return [
            (self, RDF.type, om_ns.Height),
            (self, om_ns.hasNumericalValue, Literal(self.height, datatype=XSD.float)),
            (self, om_ns.hasDimension, self.unit),
        ]
        

def make_height(height: float, unit=om_ns.millimetre, ns=strateos_ns, graph=g) -> URIRef:
    height_uri = Height(height, unit=unit, ns=ns)
    for triple in height_uri.triples():
        graph.add(triple)
    return height_uri
    
    

In [16]:
class WellVolume(URIRef):
    volume: float
    unit: URIRef
    count = 0
    def __new__(cls, *args, **kwargs):
        ns = kwargs['ns']
        URI = ns[f"wellvolume{cls.count}"]
        cls.count += 1
        return super().__new__(cls, URI)
    
    def __init__(self, volume: float, unit=om_ns.microlitre, ns=strateos_ns):
        self.volume = volume
        self.unit = unit
        super().__init__()
        
    def triples(self) -> List[Tuple[URIRef, URIRef, URIRef]]:
        return [
            (self, RDF.type, om_ns.Volume),
            (self, om_ns.hasNumericalValue, Literal(self.volume, datatype=XSD.float)),
            (self, om_ns.hasDimension, self.unit),
        ]
        

def make_well_volume(volume: float, unit=om_ns.microlitre, ns=strateos_ns, graph=g) -> URIRef:
    volume_uri = WellVolume(volume, unit=unit, ns=ns)
    for triple in volume_uri.triples():
        graph.add(triple)
    return volume_uri
    
    

In [24]:
def make_restriction(classURI: URIRef, propertyURI: URIRef, value: URIRef) -> Tuple[BNode, List[Triple]]:
    """Return a Blank node and list of tuples for a restriction definition.
    The Blank node can be used as a class specifier, and the Triples should be added to
    the graph of interest.
    """
    bnode: BNode = BNode()
    triples: List[Triple] = []
    triples.append((bnode, RDF.type, OWL.Restriction))
    triples.append((bnode, OWL.onProperty, propertyURI))
    triples.append((bnode, OWL.hasValue, value))
    triples.append((classURI, RDFS.subClassOf, bnode))
    
    return bnode, triples



In [18]:
def add_all(g: Graph, triples: List[Triple]) -> None:
    for t in triples:
        g.add(t)

In [19]:
def prefix(g: Graph) -> None:
    g.add((plate_iri, RDFS.subClassOf, r))

In [20]:

for _, x in strateos_plates.iterrows():
    id = x['Id']
    vendor = x['Vendor']
    vendor_iri = find_vendor(vendor)
    if vendor_iri is None:
        vendor_iri = add_vendor(vendor)
    catalog_number = x['Catalog Number']
    plate_iri = rdflib.term.URIRef(strateos_ns[quote(vendor+id)])
    ce = catalog_entry(vendor, catalog_number)
    g.add((plate_iri, RDFS.subClassOf, co_ns.Plate))
    _r, triples = make_restriction(plate_iri, co_ns.hasCatalogEntry, ce)
    add_all(g, triples)

    _r, triples = make_restriction(plate_iri, co_ns.wellCount, Literal(int(x['Well Count'])))
    add_all(g, triples)
    cols = int(x['Column Count'])
    rows = int(x['Well Count'] / cols)
    
    _r, triples = make_restriction(plate_iri, co_ns.columnCount, Literal(cols))
    add_all(g, triples)
    
    _r, triples = make_restriction(plate_iri, co_ns.rowCount, Literal(rows))
    add_all(g, triples)
    
    g.add((plate_iri, RDFS.comment, Literal(vendor + " " + x['Name'])))
    g.add((plate_iri, RDFS.label, Literal(id)))
    
    _r, triples = make_restriction(plate_iri, co_ns.equipmentVendor, vendor_iri)
    add_all(g, triples)
    
    well_depth = make_well_depth(float(x['Well Depth (mm)']))
    _r, triples = make_restriction(plate_iri, co_ns.wellDepth, well_depth)
    add_all(g, triples)
    
    well_vol = make_well_volume(float(x['Well Volume (ul)']))
    _r, triples = make_restriction(plate_iri, co_ns.wellVolume, well_vol)
    add_all(g, triples)    
    
    height = make_height(float(x['Height (mm)']))
    _r, triples = make_restriction(plate_iri, co_ns.height, height)
    add_all(g, triples)    
    
    print(f"Added {vendor} {id} as {plate_iri}")

Added Eppendorf 384-pcr as urn:absolute:strateos-containers#Eppendorf384-pcr
Added Mesoscale 96-10-spot-vplex-m-pro-inflamm1-MSD as urn:absolute:strateos-containers#Mesoscale96-10-spot-vplex-m-pro-inflamm1-MSD
Added Corning 384-flat-white-white-nbs as urn:absolute:strateos-containers#Corning384-flat-white-white-nbs
Added PerkinElmer 384-flat-white-white-optiplate as urn:absolute:strateos-containers#PerkinElmer384-flat-white-white-optiplate
Added Fisher 96-v-kf as urn:absolute:strateos-containers#Fisher96-v-kf
Added Chemspeed chemspeed-96-sealed-pin-rack as urn:absolute:strateos-containers#Chemspeedchemspeed-96-sealed-pin-rack
Added Labcyte 384-echo as urn:absolute:strateos-containers#Labcyte384-echo
Added Costar 96-flat-clear-costar-3590 as urn:absolute:strateos-containers#Costar96-flat-clear-costar-3590
Added Corning 96-flat-uv as urn:absolute:strateos-containers#Corning96-flat-uv
Added ThermoFisher 96-flat-white-dc as urn:absolute:strateos-containers#ThermoFisher96-flat-white-dc
Adde

In [21]:
g.serialize(format='turtle')

b'@base <urn:absolute:strateos-containers> .\n@prefix cat: <urn:absolute:strateos-containers#> .\n@prefix cat_ent: <urn:absolute:strateos_containers_catalog#> .\n@prefix cont: <https://www.dropbox.com/s/s1e2dzw64m01f9n/container-ontology.ttl#> .\n@prefix om: <http://www.ontology-of-units-of-measure.org/resource/om-2/> .\n@prefix owl: <http://www.w3.org/2002/07/owl#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\n<> a owl:ontology ;\n    rdfs:label "Strateos Catalog Containers"@en ;\n    owl:imports om:,\n        <https://www.dropbox.com/s/s1e2dzw64m01f9n/container-ontology.ttl> .\n\n<#Axygenres-mw12-hp> rdfs:label "res-mw12-hp" ;\n    rdfs:comment "Axygen Multiple Well Reagent Reservoir with 12-Channel Trough, High Profile" ;\n    rdfs:subClassOf cont:Plate ;\n    owl:subClassOf [ a owl:Restriction ;\n            owl:hasValue 12 ;\n            owl:onProperty cont:columnCount ],\n        [ a owl:Restriction ;\n          

In [22]:
g.serialize('../owl/strateos-catalog.ttl', format='turtle')